# Dream Destination Finder with CrewAI and Amazon Bedrock

In this notebook, we will explore how to use the CrewAI framework with Amazon Bedrock to build an intelligent agent that can find dream travel destinations based on user preferences. The agent will utilize a large language model (LLM) and web search capabilities to research and recommend destinations that match the user's description.

### Prerequisites

Before we begin, make sure you have the following installed:
`boto3` and `botocore` for interacting with AWS services
`crewai` and `crewai_tools` for building agentic workflows

In [17]:
# !pip install boto3==1.34.162 botocore==1.34.162 crewai==0.70.1 crewai_tools==0.12.1 duckduckgo-search==6.3.1 unstructured==0.16.6 PyPDF2==3.0.1

__!pip install boto3 botocore crewai crewai_tools duckduckgo-search langchain-community -q__

We start by importing the necessary modules from the crewai and crewai_tools packages.

#### Configuring AWS Credentials:
Before using Amazon Bedrock, ensure that your AWS credentials are configured correctly. You can set them up using the AWS CLI or by setting environment variables. For this notebook, we’ll assume that the credentials are already configured.

To use bedrock we will use [__CrewAI__ __LLM__ api](https://docs.crewai.com/how-to/llm-connections#supported-providers) 

In [2]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### Define web-search tool

In [3]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Configuring the LLM

We will use Anthropic’s Claude-3 model via Amazon Bedrock as our LLM. CrewAI uses LiteLLM under the hood to interact with different LLM providers.


In [4]:
# Configure the LLM
llm = LLM(model="bedrock/anthropic.claude-3-haiku-20240307-v1:0")

### Defining the Agent

We will create an agent with the role of a “Travel Destination Researcher.” This agent will be responsible for finding destinations that match the user’s travel preferences.

In [5]:
# Define the Agent
travel_agent = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations matching user preferences',
    backstory="You are an experienced travel agent specializing in personalized travel recommendations.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

### Defining the Task

We need to specify the task that the agent will perform. The task includes a description, expected output, and is assigned to the agent we just created.

In [6]:
# Define the Task
task = Task(
    description="Based on the user's travel preferences: {preferences}, research and recommend suitable travel destinations.",
    expected_output="A list of recommended destinations with brief descriptions.",
    agent=travel_agent
)

### Creating the Crew

A crew is a team of agents working together to achieve a common goal. In this case, we have only one agent, but the framework allows for scalability.


In [7]:
# Create the Crew
crew = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
)

### Executing the Workflow

Now, we can execute the crew with the user’s travel preferences as input.

In [8]:
# User input for travel preferences
user_input = {
    "preferences": "I want a tropical beach vacation with great snorkeling and vibrant nightlife."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To find the best tropical beach destinations with great snorkeling and vibrant nightlife, I will need to gather information about potential locations that match those criteria.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach destinations with snorkeling and nightlife\"}"
## Tool Output: 
9 Active Beach Vacations With Snorkeling, Hiking, and Endless Water Sports ... offering an active lifestyle paired with a happening nightlife scene, Here, you can golf, snorkel, ... tropical setting. Visitors can indulge in world-class snorkeling, diving among colorful coral reefs, or simply bask in the sun on powdery white sand beaches. ... Maldives. The Maldives is synonymous with luxury

#### As the crew executes, CrewAI will:

•	Decompose the task into actions using ReAct (Reasoning and Act), optionally using the tools assigned to the agent.

•	Make multiple calls to Amazon Bedrock to complete each step from the previous phase.

In [9]:
from IPython.display import Markdown

In [10]:
Markdown(result.raw)

Here are my top recommended tropical beach destinations that match your preferences for a beach vacation with excellent snorkeling and a lively nightlife scene:

1. Maldives
The Maldives is an archipelago of over 1,000 islands in the Indian Ocean, known for its stunning turquoise waters, pristine white sand beaches, and world-class snorkeling and diving. Many of the resorts offer vibrant nightlife options, from beachside bars and clubs to overwater bungalow parties.

2. Puerto Rico
The island of Puerto Rico combines beautiful tropical beaches, lush rainforests, and a bustling urban center in San Juan. Visitors can enjoy excellent snorkeling around the coral reefs, then experience the renowned nightlife and Latin-infused music scene in the capital city.

3. Phuket, Thailand
Phuket is a popular beach destination in southern Thailand, boasting stunning beaches, clear waters teeming with marine life, and a lively nightlife scene. Patong Beach in particular is known for its vibrant bars, clubs, and entertainment.

4. Cancun, Mexico
Cancun on the Yucatan Peninsula is famous for its stunning white sand beaches, turquoise waters, and excellent snorkeling opportunities. The hotel zone also offers a thriving nightlife with countless bars, clubs, and beachside parties.

5. Hawaii
The Hawaiian islands, such as Maui and the Big Island, provide incredible tropical beach settings with excellent snorkeling and diving among vibrant coral reefs. Many resort areas also have a lively nightlife with oceanview bars, live music, and luaus.

I hope these recommendations give you a great starting point to plan your dream tropical beach vacation! Let me know if you need any other details.

### Adding Memory to the Agent
CrewAI supports [several memory types](https://docs.crewai.com/concepts/memory#implementing-memory-in-your-crew), which help agents remember and learn from past interactions. In this case, we’ll enable short-term memory using Amazon Bedrock’s embedding model.

In [11]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v2:0",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

2024-11-23 13:11:41,303 - 8615497536 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [12]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Thought: Thought: To find suitable travel destinations for the user's preferences of a tropical beach vacation with great snorkeling and vibrant nightlife, I will need to search for information on destinations that match those criteria.
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach destinations with snorkeling and nightlife\"}"
## Tool Output: 
9 Active Beach Vacations With Snorkeling, Hiking, and Endless Water Sports ... offering an active lifestyle paired with a happening nightlife scene, Here, you can golf, snorkel, ... tropical setting. Visitors can indulge in world-class snorkeling, diving among colorful coral reefs, or simply bask in the sun on powdery white sand beaches. ... Maldiv

In [13]:
Markdown(result_with_memory.raw)

Recommended Tropical Beach Destinations with Great Snorkeling and Vibrant Nightlife:

1. Maldives - The Maldives is renowned for its luxury resorts, crystal-clear waters, and colorful coral reefs, making it an excellent destination for snorkeling. It also has a lively nightlife scene, with many resorts offering bars, clubs, and other entertainment options.

2. Puerto Rico - Puerto Rico is a diverse destination that combines tropical beaches, rich culture, and a vibrant nightlife. The island offers excellent snorkeling opportunities, particularly in areas like the Culebra and Vieques islands. San Juan, the capital, is known for its lively bars, clubs, and music scene.

3. Hawaii - The Hawaiian islands are a classic tropical beach destination, with world-class snorkeling in places like Maui and the Big Island. Hawaii also has a thriving nightlife, with lively bars, clubs, and live music venues, especially in areas like Waikiki Beach on Oahu.

4. Bali, Indonesia - Bali is a popular tropical destination known for its stunning beaches, rich culture, and vibrant nightlife. The island offers excellent snorkeling opportunities, particularly in the Gili Islands. Bali's nightlife is centered in areas like Seminyak and Kuta, with a wide range of bars, clubs, and beach parties.

5. Phuket, Thailand - Phuket is a tropical paradise in Thailand, offering beautiful beaches, clear waters, and abundant marine life for snorkeling. The island also boasts a lively nightlife scene, with bustling beach clubs, bars, and entertainment districts like Patong Beach.

These destinations offer the perfect combination of tropical beaches, excellent snorkeling, and vibrant nightlife to match the user's travel preferences. I hope this list provides a good starting point for planning their dream vacation.

### Integrating Retrieval-Augmented Generation (RAG) with Amazon Bedrock Knowledge Base
In this section, we will enhance our dream destination finder agent by incorporating Retrieval-Augmented Generation (RAG) using Amazon Bedrock’s Knowledge Base. This will allow our agent to access up-to-date and domain-specific travel information, improving the accuracy and relevance of its recommendations.



#### What is Retrieval-Augmented Generation (RAG)?

RAG is a technique that combines the capabilities of large language models (LLMs) with a retrieval mechanism to fetch relevant information from external data sources. By integrating RAG, our agent can retrieve the most recent and specific information from a knowledge base, overcoming the limitations of LLMs that may have outdated or insufficient data.

Setting Up Amazon Bedrock Knowledge Base

Before we proceed, ensure you have access to Amazon Bedrock and the necessary permissions to create and manage knowledge bases.

* Step 1: Prepare Your Data
* Step 2: Create a Knowledge Base in Amazon Bedrock
* Step 3: Note the Knowledge Base ID

After the knowledge base is created, note down its Knowledge Base ID (kb_id), which will be used in our code.

Updating the Agent to Use RAG with CrewAI

We will modify our agent to include a custom tool that queries the Amazon Bedrock Knowledge Base. This allows the agent to retrieve up-to-date information during its reasoning process.

### FAIS Vector Store Set up:

In [16]:
import os
from uuid import uuid4
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings

documents = []
pdf_folder = '/aim323_build_agents_with_bedrock_oss/data/travel_guides'

# Loop through PDFs in the specified folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        file_path = os.path.join(pdf_folder, pdf_file)
        
        # Extract text from PDF
        reader = PdfReader(file_path)
        text_content = ""
        for page in reader.pages:
            text_content += page.extract_text() + "\n"
        
        # Create a Document instance
        doc = Document(
            page_content=text_content.strip(),
            metadata={}  # Leave metadata empty for now
        )
        documents.append(doc)

# Initialize FAISS vector store and embeddings
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
vector_store = FAISS.from_documents(documents, embeddings)

# Add unique IDs to documents and save the vector store
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['4384dfd1-6f05-4bc6-80c1-999096fbee02',
 '4f678862-41ec-4aa2-aa19-c604bdab99b7',
 '656067da-4841-47ba-8541-af7dab43768b',
 'a0662ae0-4bf9-4bb8-8e87-85f173a4ab16',
 '0067e09c-f803-4cf7-84ae-04c4b83b034b',
 '58ac4057-5464-44c2-8c92-70c0476a5344',
 '7bad175c-3184-4c74-b000-283aef8e8b06',
 'a5141eeb-b463-4338-b32e-34adf8cfe1f2',
 'd1bab26b-dcca-4e37-ad9f-2acdbd8bf366',
 '0489fd93-8b89-42d6-a014-98a4b2436254',
 'd2a2f1d9-7a9d-4d32-9068-f053f2268bf1',
 'a211ac2a-9290-45b6-849b-cb9a213f54f7',
 '20a77c93-9466-4c9d-a98d-7a95b13828b1']

In [17]:
@tool("TravelExpertSearchEngine")
def query_knowledge_base(question: str) -> str:
    """Queries the Amazon Bedrock Knowledge Base for travel-related information."""
    try:
        res = vector_store.similarity_search(
        question,
        k=1,
        )        
        return res[0].page_content
    except KeyError:
        return "No data available"



### Update the Agent with the New Tool
We will update our agent to include the TravelExpert tool.

In [18]:
# Update the Agent
agent_with_rag = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations in the USA using only the travel guide available, first lookup cities using the tool to match user preferences and then use information from the search engine, nothing else.',
    backstory="""You are an experienced travel agent specializing in personalized travel recommendations. 
                 Your approach is as follows: 
                 Deduce which regions within the USA will have those activities listed by the user.
                 List major cities within that region
                 Only then use the tool provided to look up information, look up should be done by passing city highlights and activities.
                 Only suggest places that were extracted using the lookup tool,
              """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[query_knowledge_base],  # Include the RAG tool
    max_iter=3
)


### Update the task and set up the Crew

In [19]:
# Define the Task
task_with_rag = Task(
    description="Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: {preferences}",
    expected_output="A place where they can travel to along with recommendations on what to see and do while there.",
    agent=agent_with_rag
)


# Create the Crew
crew_with_rag = Crew(
    agents=[agent_with_rag],
    tasks=[task_with_rag],
    verbose=True,
)

2024-11-23 13:15:37,625 - 8615497536 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
# User input for travel preferences
user_input = {
    "preferences": "Where can I go for cowboy vibes, watch a rodeo, and a visit museums?"
}

# Execute the Crew
result_with_rag = crew_with_rag.kickoff(inputs=user_input)


# Agent: Travel Destination Researcher
## Task: Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: Where can I go for cowboy vibes, watch a rodeo, and a visit museums?


# Agent: Travel Destination Researcher
## Thought: Thought: To find the best travel destinations for the user's request, I will first need to determine which regions in the USA would have the activities they are interested in - cowboy vibes, watching a rodeo, and visiting museums.
## Using tool: TravelExpertSearchEngine
## Tool Input: 
"{\"question\": \"regions in the USA with cowboy culture, rodeos, and museums\"}"
## Tool Output: 
Travel Guide: Dallas
Generated by Llama3.1 405B
 
Dallas, the vibrant heart of Texas, is a city that captivates with its dynamic blend of modern
sophistication and cowboy charm. As the ninth-largest city in the United States, Dallas dazzles vi

### Display the results

In [21]:
# Display the result
Markdown(result_with_rag.raw)

Based on the user's request for travel destinations with cowboy vibes, rodeos, and museums, I recommend visiting Dallas, Texas. 

Dallas offers a unique blend of modern sophistication and classic Texas cowboy charm. Here are some of the top things to do in Dallas:

Cowboy Culture & Rodeos:
- Explore the lively honky-tonk bars and music venues in the Deep Ellum neighborhood, which has a thriving cowboy/Western culture.
- Attend a professional rodeo event, such as the annual State Fair of Texas Rodeo, to experience the excitement of bull riding, team roping, and other classic rodeo events.

Museums & Arts:
- Visit the Sixth Floor Museum, which chronicles the life and assassination of President John F. Kennedy.
- Discover the Dallas Arts District, a 68-acre cultural hub featuring renowned institutions like the Winspear Opera House and the Nasher Sculpture Center.
- Explore the impressive collection of modern and contemporary art at the Nasher Sculpture Center.

Additional Attractions:
- Take in panoramic views of the Dallas skyline from the observation deck of the iconic Reunion Tower.
- Hike the scenic trails of the Arbor Hills Nature Preserve or visit Dinosaur Valley State Park to see the footprints of ancient dinosaurs.
- Enjoy mouthwatering Tex-Mex cuisine at local favorites like Meso Maya.

With its perfect blend of cowboy culture, rodeo events, world-class museums, and outdoor adventures, Dallas offers the quintessential Texas experience. It's an ideal destination for the user's travel preferences.